# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [50]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [57]:
weather_data = pd.read_csv("../starter_code/city_data.csv")

weather_df = pd.DataFrame(weather_data)
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,Rāy,23.6833,85.0500,82.4,36,1,2.30
1,1,Cape Town,-33.9258,18.4232,64.0,82,0,9.22
2,2,Saldanha,-33.0117,17.9442,59.0,67,0,5.75
3,3,Souillac,-20.5167,57.5167,82.4,83,75,11.50
4,4,Bredasdorp,-34.5322,20.0403,50.0,93,6,4.61


In [58]:
gmaps.configure(api_key = g_key)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [60]:
locations = weather_df[['Latitude', 'Longitude']]

humidity = weather_df['Humidity'].astype(float)

In [64]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, 
                                    max_intensity=100, point_radius=3.0)

fig.add_layer(heatmap_layer)

plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [67]:
ideal_weather_df = weather_data.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Temperature"] >= 70) & (weather_df["Temperature"] <= 80)].dropna()

ideal_weather_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
27,27,Cabedelo,-6.9811,-34.8339,75.20,100,0,3.44
131,131,Makkah al Mukarramah,21.4267,39.8261,77.43,54,0,1.43
223,223,Aswān,24.0934,32.9070,76.33,19,0,5.77


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [89]:
ideal_weather_df['Hotel Name'] = ""

ideal_weather_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
27,27,Cabedelo,-6.9811,-34.8339,75.20,100,0,3.44,
131,131,Makkah al Mukarramah,21.4267,39.8261,77.43,54,0,1.43,
223,223,Aswān,24.0934,32.9070,76.33,19,0,5.77,


In [102]:
for index, row in ideal_weather_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }

        lat = row['Latitude']
        lng = row['Longitude']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        ideal_weather_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        ideal_weather_df.loc[index, "Hotel Name"] = "NaN"
        
ideal_weather_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
27,27,Cabedelo,-6.9811,-34.8339,75.20,100,0,3.44,Hotel Almagre
131,131,Makkah al Mukarramah,21.4267,39.8261,77.43,54,0,1.43,Pullman Zamzam Makkah
223,223,Aswān,24.0934,32.9070,76.33,19,0,5.77,Sofitel Legend Old Cataract Aswan


In [114]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = ideal_weather_df[["Latitude", "Longitude"]]

In [115]:
# Add marker layer ontop of heat map
# Display figure

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))